In [62]:
import sys

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

sys.path.append("../../..")
from utils.get_date_today_iso_string import get_date_today_iso_string
from notebooks.checking_functions import (
    null_value_check,
    whitespaces_check,
    null_or_negative_value_check,
    duplicates_check,
    url_format_and_null_check,
    datestring_format_and_null_check
)



spark = SparkSession.builder.appName("ReadParquetFile").getOrCreate()

extraction_date = get_date_today_iso_string()

bronze_to_silver_df = spark.read.parquet(f"../../../data/bronze/coin_markets/parquet/{extraction_date}/")

with open("../../bronze/schema_order.txt") as f:
    bronze_to_silver_df_column_order = f.read().split(",")

bronze_to_silver_df = bronze_to_silver_df.select(*bronze_to_silver_df_column_order)


In [63]:
# CHECK FOR NULL VALUES, UNWANTED WHITESPACES AND DUPLICATES OF ID COLUMN

null_value_check(bronze_to_silver_df, "id")
whitespaces_check(bronze_to_silver_df, "id")
duplicates_check(bronze_to_silver_df, "id")

+---+-------+
| id|null_id|
+---+-------+
+---+-------+

+---+--------------+
| id|whitespaced_id|
+---+--------------+
+---+--------------+

+----------+-------------------------+
|cleaned_id|cleaned_id_distinct_count|
+----------+-------------------------+
+----------+-------------------------+



In [64]:
# CHECK FOR NULL VALUES abd UNWANTED WHITESPACES OF SYMBOL COLUMN

null_value_check(bronze_to_silver_df, "symbol")
whitespaces_check(bronze_to_silver_df, "symbol")

+---+-----------+
| id|null_symbol|
+---+-----------+
+---+-----------+

+---+------------------+
| id|whitespaced_symbol|
+---+------------------+
+---+------------------+



In [65]:
# CHECK FOR NULL VALUES abd UNWANTED WHITESPACES OF NAME COLUMN

null_value_check(bronze_to_silver_df, "name")
whitespaces_check(bronze_to_silver_df, "name")

+---+---------+
| id|null_name|
+---+---------+
+---+---------+

+---+----------------+
| id|whitespaced_name|
+---+----------------+
+---+----------------+



In [68]:
# CHECK URL FORMAT AND NULL VALUES FOR IMAGE COLUMN

url_format_and_null_check(bronze_to_silver_df, "image")

+---+-----------------+
| id|invalid_image_url|
+---+-----------------+
+---+-----------------+



In [69]:
# CHECK FOR NEGATIVE OR NULL VALUES OF CURRENT PRICE COLUMN

null_or_negative_value_check(bronze_to_silver_df,"current_price")


+---+---------------------+
| id|invalid_current_price|
+---+---------------------+
+---+---------------------+



In [70]:
# CHECK FOR NEGATIVE OR NULL VALUES OF MARKET CAP COLUMN

null_or_negative_value_check(bronze_to_silver_df,"market_cap")

+---+------------------+
| id|invalid_market_cap|
+---+------------------+
+---+------------------+



In [71]:
# CHECK FOR NEGATIVE OR NULL VALUES OF FULLY DILUATED VALUATION COLUMN

null_or_negative_value_check(bronze_to_silver_df,"fully_diluted_valuation")

+---+-------------------------------+
| id|invalid_fully_diluted_valuation|
+---+-------------------------------+
+---+-------------------------------+



In [72]:
# CHECK FOR NEGATIVE OR NULL VALUES OF TOTAL VOLUME COLUMN

null_or_negative_value_check(bronze_to_silver_df,"total_volume")

+---+--------------------+
| id|invalid_total_volume|
+---+--------------------+
+---+--------------------+



In [73]:
# CHECK FOR NEGATIVE OR NULL VALUES OF HIGH 24H COLUMN

null_or_negative_value_check(bronze_to_silver_df,"high_24h")

+---+----------------+
| id|invalid_high_24h|
+---+----------------+
+---+----------------+



In [74]:
# CHECK FOR NEGATIVE OR NULL VALUES OF LOW 24H COLUMN

null_or_negative_value_check(bronze_to_silver_df,"low_24h")

+---+---------------+
| id|invalid_low_24h|
+---+---------------+
+---+---------------+



In [ ]:
# CHECK FOR INVALID HIGH AND LOW PRICES AT 24H WHEREIN THE HIGH IS LOWER THAN LOW

bronze_to_silver_df.filter(F.col("high_24h") < F.col("low_24h"))\
    .select(F.col("id"),F.col("high_24h"), F.col("low_24h"))\
    .show()

+---+--------+-------+
| id|high_24h|low_24h|
+---+--------+-------+
+---+--------+-------+



In [75]:
# CHECK FOR NULL VALUES OF PRICE CHANGE 24H COLUMN

null_value_check(bronze_to_silver_df,"price_change_24h")

+---+---------------------+
| id|null_price_change_24h|
+---+---------------------+
+---+---------------------+



In [76]:
# CHECK FOR NULL VALUES OF PRICE CHANGE PERCENTAGE 24H COLUMN

null_value_check(bronze_to_silver_df,"price_change_percentage_24h")

+---+--------------------------------+
| id|null_price_change_percentage_24h|
+---+--------------------------------+
+---+--------------------------------+



In [141]:
# CHECK IF PRICE CHANGE PERCENTAGE IS EQUAL TO PRICE CHANGE DIVIDED BY PREVIOUS PRICE WITH TOLERANCE OF 0.01

bronze_to_silver_df.withColumn("previous_price", F.col("current_price") - F.col("price_change_24h"))\
.withColumn("derived_price_change_percentage_24h", (F.col("price_change_24h") / F.col("previous_price") * 100))\
.filter(F.abs((F.col("derived_price_change_percentage_24h"))) - F.abs((F.col("price_change_percentage_24h"))) > 0.01)\
.select(F.col("id"),F.col("current_price"),F.col("price_change_24h"),F.col("previous_price"),\
        F.col("derived_price_change_percentage_24h"),F.col("price_change_percentage_24h"))\
.show()

+-----------------+-------------+----------------+--------------+-----------------------------------+---------------------------+
|               id|current_price|price_change_24h|previous_price|derived_price_change_percentage_24h|price_change_percentage_24h|
+-----------------+-------------+----------------+--------------+-----------------------------------+---------------------------+
|         polkadot|         3.27|        0.463976|      2.806024|                 16.534997562387208|                   16.50579|
|internet-computer|         9.72|            1.66|          8.06|                 20.595533498759302|                   20.56348|
|     render-token|         2.33|        0.282552|      2.047448|                 13.800203961223922|                   13.78276|
+-----------------+-------------+----------------+--------------+-----------------------------------+---------------------------+



In [105]:
# CHECK FOR NULL VALUES OF MARKET CAP CHANGE 24H COLUMN

null_value_check(bronze_to_silver_df,"market_cap_change_24h")

+---+--------------------------+
| id|null_market_cap_change_24h|
+---+--------------------------+
+---+--------------------------+



In [106]:
# CHECK FOR NULL VALUES OF MARKET CAP CHANGE PERCENTAGE 24H COLUMN

null_value_check(bronze_to_silver_df,"market_cap_change_percentage_24h")

+---+-------------------------------------+
| id|null_market_cap_change_percentage_24h|
+---+-------------------------------------+
+---+-------------------------------------+



In [79]:
# CHECK FOR NEGATIVE OR NULL VALUES OF CIRCULATING SUPPLY COLUMN

null_or_negative_value_check(bronze_to_silver_df,"circulating_supply")

+---+--------------------------+
| id|invalid_circulating_supply|
+---+--------------------------+
+---+--------------------------+



In [80]:
# CHECK FOR NEGATIVE OR NULL VALUES OF TOTAL SUPPLY COLUMN

null_or_negative_value_check(bronze_to_silver_df,"total_supply")

+---+--------------------+
| id|invalid_total_supply|
+---+--------------------+
+---+--------------------+



In [81]:
# CHECK FOR NEGATIVE OR NULL OF MAX SUPPLY COLUMN

null_or_negative_value_check(bronze_to_silver_df,"max_supply")

+--------------------+------------------+
|                  id|invalid_max_supply|
+--------------------+------------------+
|            ethereum|              NULL|
|              tether|              NULL|
|              solana|              NULL|
|            usd-coin|              NULL|
|        staked-ether|              NULL|
|                tron|              NULL|
|            dogecoin|              NULL|
|        figure-heloc|              NULL|
|       wrapped-steth|              NULL|
|  wrapped-beacon-eth|              NULL|
|                usds|              NULL|
|        wrapped-eeth|              NULL|
|binance-bridged-u...|              NULL|
|         ethena-usde|              NULL|
|           leo-token|              NULL|
|coinbase-wrapped-btc|              NULL|
|                weth|              NULL|
|              monero|              NULL|
|           shiba-inu|              NULL|
|            polkadot|              NULL|
+--------------------+------------

In [82]:
# CHECK FOR NEGATIVE OR NULL VALUES OF ath COLUMN

null_or_negative_value_check(bronze_to_silver_df,"ath")

+---+-----------+
| id|invalid_ath|
+---+-----------+
+---+-----------+



In [83]:
# CHECK FOR NULL VALUES OF ath_change_percentage COLUMN

null_value_check(bronze_to_silver_df,"ath_change_percentage")

+---+--------------------------+
| id|null_ath_change_percentage|
+---+--------------------------+
+---+--------------------------+



In [84]:
# CHECK FOR INVALID DATE FOIRMAT AND NULL VALUES OF ath_date COLUMN

datestring_format_and_null_check(bronze_to_silver_df,"ath_date")

+---+--------------------+
| id|invalid_ath_date_url|
+---+--------------------+
+---+--------------------+



In [85]:
# CHECK FOR NEGATIVE OR NULL VALUES OF atl COLUMN

null_or_negative_value_check(bronze_to_silver_df,"atl")

+---+-----------+
| id|invalid_atl|
+---+-----------+
+---+-----------+



In [86]:
# CHECK FOR NULL VALUES OF atl_change_percentage COLUMN

null_value_check(bronze_to_silver_df,"atl_change_percentage")

+---+--------------------------+
| id|null_atl_change_percentage|
+---+--------------------------+
+---+--------------------------+



In [87]:
# CHECK FOR INVALID DATE FOIRMAT AND NULL VALUES OF atl_date COLUMN

datestring_format_and_null_check(bronze_to_silver_df,"atl_date")

+---+--------------------+
| id|invalid_atl_date_url|
+---+--------------------+
+---+--------------------+



In [88]:
bronze_to_silver_df.dtypes

[('id', 'string'),
 ('symbol', 'string'),
 ('name', 'string'),
 ('image', 'string'),
 ('current_price', 'double'),
 ('market_cap', 'bigint'),
 ('market_cap_rank', 'bigint'),
 ('fully_diluted_valuation', 'bigint'),
 ('total_volume', 'double'),
 ('high_24h', 'double'),
 ('low_24h', 'double'),
 ('price_change_24h', 'double'),
 ('price_change_percentage_24h', 'double'),
 ('market_cap_change_24h', 'double'),
 ('market_cap_change_percentage_24h', 'double'),
 ('circulating_supply', 'double'),
 ('total_supply', 'double'),
 ('max_supply', 'double'),
 ('ath', 'double'),
 ('ath_change_percentage', 'double'),
 ('ath_date', 'string'),
 ('atl', 'double'),
 ('atl_change_percentage', 'double'),
 ('atl_date', 'string'),
 ('roi', 'struct<currency:string,percentage:double,times:double>'),
 ('last_updated', 'string')]

In [89]:
# CHECK FOR PARTIALLY EMPTY STRUCT OR WHITESPACED currency value on roi column

bronze_to_silver_df.filter(F.col("roi").isNotNull()
    & 
    (
        F.col("roi.currency").isNull() |
        (F.trim(F.col("roi.currency")) != F.col("roi.currency")) |
        F.col("roi.percentage").isNull() |
        F.col("roi.times").isNull()
    )
    )\
    .select(F.col("id"), F.col("roi.currency"), F.col("roi.percentage"), F.col("roi.times")).show(truncate=False)

+---+--------+----------+-----+
|id |currency|percentage|times|
+---+--------+----------+-----+
+---+--------+----------+-----+



In [90]:
# CHECK FOR INVALID DATE FOIRMAT AND NULL VALUES OF last_updated COLUMN

datestring_format_and_null_check(bronze_to_silver_df,"last_updated")

+---+------------------------+
| id|invalid_last_updated_url|
+---+------------------------+
+---+------------------------+



In [91]:
# silver_df = (
#     bronze_df
#     .dropDuplicates()
#     .filter(bronze_df["price"].isNotNull())
#     .withColumnRenamed("timestamp", "event_time")
# )

# silver_df.write.mode("overwrite").parquet("silver/crypto_data.parquet")

In [94]:
bronze_to_silver_df\
.select(F.col("id"),F.col("current_price"),F.col("market_cap"),F.col("price_change_24h"),F.col("price_change_percentage_24h"))\
.show(5)

+-----------+-------------+-------------+--------------------+---------------------------+
|         id|current_price|   market_cap|    price_change_24h|price_change_percentage_24h|
+-----------+-------------+-------------+--------------------+---------------------------+
|    bitcoin|     102137.0|2037445236754|              231.81|                    0.22748|
|   ethereum|      3436.94| 414845840815|               82.37|                    2.45545|
|     tether|     0.999495| 183355307131|-1.57802736198365E-4|                   -0.01579|
|     ripple|         2.32| 139316173499|            0.092612|                     4.1631|
|binancecoin|       993.43| 136772299019|               27.97|                    2.89709|
+-----------+-------------+-------------+--------------------+---------------------------+
only showing top 5 rows


In [ ]:
df.filter(F.col(column_name).isNull())\
    .select(F.col("id"),F.col(column_name).alias(f"null_{column_name}"))\
    .show()

101